<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/convert_sbert_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers memory_profiler datasets
import time
import datetime
tic = time.time()

In [3]:
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
c = "https://raw.githubusercontent.com/jinhangjiang/Medium_Demo/main/Data2vec_vs_SBERT/bbc-text.csv"
df = pd.read_csv(c)

In [5]:
print(df.category.unique())
print(pd.factorize(df.category)[1])

['tech' 'business' 'sport' 'entertainment' 'politics']
Index(['tech', 'business', 'sport', 'entertainment', 'politics'], dtype='object')


In [5]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)

In [6]:
def TextClassification_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int, metrics_name: str):

  # Make data
  X = Data
  y = Target
  y = pd.factorize(y)[0]

  # Load Metrics
  metric = load_metric(metrics_name)

  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Call the Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

  # Encode the text
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          return item

      def __len__(self):
          return len(self.labels)

  # convert our tokenized data into a torch Dataset
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Call Model
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda")

  # Create Metrics
  def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # 'micro', 'macro', etc. are for multi-label classification. If you are running a binary classification, leave it as default or specify "binary" for average
    return metric.compute(predictions=predictions, references=labels, average="micro")  

  # Specifiy the arguments for the trainer  
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=8,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = metrics_name,    # select the base metrics
      logging_steps=200,               # log & save weights each logging_steps
      save_steps=200,
      evaluation_strategy="steps",     # evaluate each `logging_steps`
  ) 

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()
  
  # Call the summary
  trainer.evaluate()



  return trainer, model

In [ ]:
%%time
%%memit
sbert_trainer, sbert_model = TextClassification_with_Transformer(model_name = 'sentence-transformers/all-mpnet-base-v2', 
                                                                 Data = df.text, 
                                                                 Target = df.category, 
                                                                 test_size = 0.33, 
                                                                 max_length = 512, 
                                                                 num_labels = 5, 
                                                                 num_epochs = 5, 
                                                                 metrics_name='f1')

In [7]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("swiss_judgment_prediction", "de")


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
DATA_swiss = pd.DataFrame(dataset['train'])


In [9]:
DATA_swiss_test = pd.DataFrame(dataset['test'])

In [10]:
dat= pd.concat([DATA_swiss,DATA_swiss_test])

In [22]:
dat

,id,year,text,label,language,region,canton,legal area,source_language
0,2,2000,A.- Der 1955 geborene V._ war seit 1. Septembe...,0,de,Zürich,zh,insurance law,n/a
1,3,2000,"Ansprüche nach OHG, hat sich ergeben: A.- X._ ...",1,de,Central Switzerland,lu,public law,n/a
2,4,2000,Art. 4 aBV (Strafverfahren wegen falschen Zeug...,0,de,Northwestern Switzerland,ag,public law,n/a
3,5,2000,"Art. 5 Ziff. 1 EMRK (Haftentlassung), hat sich...",1,de,n/a,n/a,public law,n/a
4,6,2000,"Mietvertrag, hat sich ergeben: A.- Die CT Cond...",0,de,n/a,n/a,civil law,n/a
...,...,...,...,...,...,...,...,...,...
9720,71527,2020,"Sachverhalt: A. B._ (Beklagter, Beschwerdegegn...",0,de,Zürich,zh,civil law,n/a
9721,71529,2020,Sachverhalt: A. Auf Anzeige von A._ hin führte...,0,de,Espace Mittelland,be,penal law,n/a
9722,71530,2020,Sachverhalt: A. Am 4. April 2013 und am 19. Ju...,0,de,Federation,ch,public law,n/a
9723,71531,2020,Sachverhalt: A. Die Staatsanwaltschaft wirft A...,0,de,Northwestern Switzerland,ag,penal law,n/a


In [ ]:
%%time
%%memit
sbert_trainer, sbert_model = TextClassification_with_Transformer(model_name = 'sentence-transformers/all-mpnet-base-v2', 
                                                                 Data = dat.text, 
                                                                 Target = dat.label, 
                                                                 test_size = 0.33, 
                                                                 max_length = 512, 
                                                                 num_labels = 2, 
                                                                 num_epochs = 5, 
                                                                 metrics_name='f1')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  if __name__ == '__main__':
Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetFor

Step,Training Loss,Validation Loss,F1
200,0.565800,0.535142,0.773121
400,0.551900,0.535556,0.773121


***** Running Evaluation *****
  Num examples = 14911
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 14911
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 14911
  Batch size = 20
